In [ ]:
# importing the libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Avoid to truncating rows and column as well

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# Import the dataset

df = pd.read_csv('gurgaon_properties_outlier_treated.csv')

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

### Built up area

In [ ]:
sns.scatterplot(x = df['built_up_area'], y = df['super_built_up_area'])

In [ ]:
sns.scatterplot(x = df['built_up_area'], y = df['carpet_area'])

In [ ]:
# checking for all the three area's are missing in a column or not

((df['super_built_up_area'].isnull()) & (df['built_up_area'].isnull()) & (df['carpet_area'].isnull()))

In [ ]:
# Extract that all rows where all the area's are present

all_present_df = df[~((df['super_built_up_area'].isnull()) | (df['built_up_area'].isnull()) | (df['carpet_area'].isnull()))]

In [ ]:
all_present_df.shape

In [ ]:
all_present_df.shape

In [ ]:
# calculate the ratio of super_built_up_area and built_up_area

super_to_built_up_ratio = (all_present_df['super_built_up_area']/all_present_df['built_up_area']).median()

In [ ]:
# calculate the ratio of carpet_area and built_up_area


carpet_to_built_up_ratio = (all_present_df['carpet_area']/all_present_df['built_up_area']).median()

In [ ]:
print(super_to_built_up_ratio, carpet_to_built_up_ratio)

In [ ]:
# both present built up null

sbc_df = df[~(df['super_built_up_area'].isnull()) & (df['built_up_area'].isnull()) & ~(df['carpet_area'].isnull())]

In [ ]:
sbc_df.head()

In [ ]:


sbc_df['built_up_area'].fillna(round(((sbc_df['super_built_up_area']/1.105) + (sbc_df['carpet_area']/0.9))/2),inplace=True)

In [ ]:
df.update(sbc_df)

In [ ]:
df.isnull().sum()

In [ ]:
# sb present c is null built up null


sb_df = df[~(df['super_built_up_area'].isnull()) & (df['built_up_area'].isnull()) & (df['carpet_area'].isnull())]

In [ ]:
sb_df.head()

In [ ]:
sb_df['built_up_area'].fillna(round(sb_df['super_built_up_area']/1.105),inplace=True)

In [ ]:
df.update(sb_df)

In [ ]:
df.isnull().sum()

In [ ]:
# sb null c is present built up null


c_df = df[(df['super_built_up_area'].isnull()) & (df['built_up_area'].isnull()) & ~(df['carpet_area'].isnull())]

In [ ]:
c_df.head()

In [ ]:
c_df['built_up_area'].fillna(round(c_df['carpet_area']/0.9),inplace=True)

In [ ]:
df.update(c_df)

In [ ]:
df.isnull().sum()

In [ ]:
sns.scatterplot(x = df['built_up_area'], y = df['price'])

In [ ]:
# find the rows where built_up_area < 2000 and price > 2.5cr

anamoly_df = df[(df['built_up_area'] < 2000) & (df['price'] > 2.5)][['price','area','built_up_area']]

In [ ]:
anamoly_df.sample(5)

In [ ]:
anamoly_df['built_up_area'] = anamoly_df['area']

In [ ]:
df.update(anamoly_df)

In [ ]:
sns.scatterplot( x = df['built_up_area'], y = df['price'])

In [ ]:
# droping these columns

df.drop(columns=['area','areaWithType','super_built_up_area','carpet_area'],inplace=True)

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

### floorNum

In [ ]:
df[df['floorNum'].isnull()]

In [ ]:
df[df['property_type'] == 'house']['floorNum'].median()

In [ ]:
df['floorNum'].fillna(2.0,inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
1011/df.shape[0]

### facing

In [ ]:
df['facing'].isnull().sum()

In [ ]:
df['facing'].value_counts().plot(kind='pie',autopct='%0.2f%%')

In [ ]:
df.drop(columns=['facing'],inplace=True)

In [ ]:
df.sample(5)

In [ ]:
df.isnull().sum()

In [ ]:
df[df['society'].isnull()]

In [ ]:
df.drop(index=[402],inplace=True)

In [ ]:
df.isnull().sum()

### agePossession

In [ ]:
df['agePossession'].value_counts()

In [ ]:
df[df['agePossession'] == 'Undefined']

In [ ]:
# checking for all the undefined value present in age_possession and try to replace with mode of the values with the help of property type and sector no.


def mode_based_imputation(row):
    if row['agePossession'] == 'Undefined':
        mode_value = df[(df['sector'] == row['sector']) & (df['property_type'] == row['property_type'])]['agePossession'].mode()
        # If mode_value is empty (no mode found), return NaN, otherwise return the mode
        if not mode_value.empty:
            return mode_value.iloc[0] 
        else:
            return np.nan
    else:
        return row['agePossession']

In [ ]:
df['agePossession'] = df.apply(mode_based_imputation,axis=1)

In [ ]:
df['agePossession'].value_counts()

In [ ]:



def mode_based_imputation2(row):
    if row['agePossession'] == 'Undefined':
        mode_value = df[(df['sector'] == row['sector'])]['agePossession'].mode()
        # If mode_value is empty (no mode found), return NaN, otherwise return the mode
        if not mode_value.empty:
            return mode_value.iloc[0] 
        else:
            return np.nan
    else:
        return row['agePossession']

In [ ]:
df['agePossession'] = df.apply(mode_based_imputation2,axis=1)

In [ ]:
df['agePossession'].value_counts()

In [ ]:
def mode_based_imputation3(row):
    if row['agePossession'] == 'Undefined':
        mode_value = df[(df['property_type'] == row['property_type'])]['agePossession'].mode()
        # If mode_value is empty (no mode found), return NaN, otherwise return the mode
        if not mode_value.empty:
            return mode_value.iloc[0] 
        else:
            return np.nan
    else:
        return row['agePossession']

In [ ]:
df['agePossession'] = df.apply(mode_based_imputation3,axis=1)

In [ ]:
df['agePossession'].value_counts()

In [ ]:
df.isnull().sum()

In [ ]:
# Export this dataset for further process

df.to_csv('gurgaon_properties_missing_value_imputation.csv',index=False)

In [ ]:
df.shape